# Convolution Neural Network

In [6]:
import deeplake
import numpy as np

import tqdm
import random
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from collections import OrderedDict

In [2]:
print(torch.__version__)

2.1.0


In [7]:
class Levi_Hassner(nn.Module):
    def __init__(self,output=2,deformable=False) -> None:
        super().__init__()
        self.deformable=deformable

        self.layers=nn.Sequential(OrderedDict([
            # first convolutional layer
            ('conv1',nn.Conv2d(3, 96, 7, padding='valid', stride=4)),  # No padding
            ('relu1',nn.ReLU()),
            ('maxpool1',nn.MaxPool2d(3, stride=2)),  # Max pooling over a (3, 3) window with 2 pixel stride)
            ('lrn1',nn.LocalResponseNorm(size=5, k=2, alpha=10**(-4), beta=0.75)),

            # second convolutional layer
            ('conv2',nn.Conv2d(96, 256, 5, padding='same')), # Same padding
            ('relu2',nn.ReLU()),
            ('maxpool2',nn.MaxPool2d(3, stride=2)),  # Max pooling over a (3, 3) window with 2 pixel stride)
            ('lrn2',nn.LocalResponseNorm(size=5, k=2, alpha=10**(-4), beta=0.75)),

            # third convolutional layer
            ('conv3',nn.Conv2d(256, 384, 3, padding='same')),  # Same padding
            ('relu3',nn.ReLU()),
            ('maxpool3',nn.MaxPool2d(3, stride=2)),  # Max pooling over a (3, 3) window with 2 pixel stride)
            ('flatten',nn.Flatten()),

            ('fc1',nn.Linear(384*6*6, 512)), # input 384 * 6 * 6 = 13824, output 512
            ('relu4',nn.ReLU()),
            ('dropout1',nn.Dropout(0.5)),

            ('fc2',nn.Linear(512,512)),
            ('relu5',nn.ReLU()),
            ('dropout2',nn.Dropout(0.5)),
            
            ('fc3',nn.Linear(512,output)), # output = number of classes 
        ]))
        self.prob=nn.Softmax

    def forward(self,x):
        x=self.layers(x)
        prob=self.prob(x)
        return prob

In [9]:
class Levi_Hassner(nn.Module):
    def __init__(self, output=2, deformable=False):
        super().__init__()
        self.deformable = deformable
        self.lrn = nn.LocalResponseNorm(size=5, k=2, alpha=10**(-4), beta=0.75)
        self.maxpool = nn.MaxPool2d(3, stride=2)  # Max pooling over a (3, 3) window with 2 pixel stride
        
        # first convolutional layer
        self.conv1 = nn.Conv2d(3, 96, 7, padding='valid', stride=4)  # No padding
        # second convolutional layer
        self.conv2 = nn.Conv2d(96, 256, 5, padding='same')  # Same padding        
        # third convolutional layer
        self.conv3 = nn.Conv2d(256, 384, 3, padding='same')  # Same padding

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(384*6*6, 512) # input 384 * 6 * 6 = 13824, output 512
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512,512)
        self.fc3 = nn.Linear(512, output) # output = number of classes 
        self.prob = nn.Softmax()

    def forward(self, x):      
        conv1 = torch.relu(self.conv1(x)) # output dim: 96 * 56 * 56
        # conv1 = conv1 if not deformable else deform_conv2d(x, offset, we)
        pool1 = self.maxpool(conv1) # output dim: 96 * 27 * 27
        norm1 = self.lrn(pool1)
        
        conv2 = torch.relu(self.conv2(norm1)) # output dim: 256 * 27 * 27
        pool2 = self.maxpool(conv2) # output dim: 256 * 13 * 13
        norm2 = self.lrn(pool2)

        conv3 = torch.relu(self.conv2(norm2)) # output dim: 384 * 13 * 13
        pool3 = self.maxpool(conv3) # output dim: 384 * 6 * 6

        flatten = self.flatten(pool3) # fc layer operates on flattened inputs
        fc1 = torch.relu(self.fc1(flatten))
        dropout1 = self.dropout(fc1)

        fc2 = torch.relu(self.fc2(dropout1))
        dropout2 = self.dropout(fc2) 

        fc3 = self.fc3(dropout2)
        prob = self.prob(fc3)
        return prob

In [4]:
gender_model = Levi_Hassner()
gender_model

Levi_Hassner(
  (lrn): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=valid)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=13824, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=2, bias=True)
  (prob): Softmax(dim=None)
)

In [8]:
Levi_Hassner()

Levi_Hassner(
  (layers): Sequential(
    (conv1): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=valid)
    (relu1): ReLU()
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (lrn1): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (relu2): ReLU()
    (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (lrn2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (relu3): ReLU()
    (maxpool3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (fc1): Linear(in_features=13824, out_features=512, bias=True)
    (relu4): ReLU()
    (dropout1): Dropout(p=0.5, inplace=False)
    (fc2): Linear(in_features=512, out_features=512, bias=True)
    (relu5): ReLU()
    (

In [10]:
age_model = Levi_Hassner(output=8)
age_model

Levi_Hassner(
  (lrn): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=valid)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=13824, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=8, bias=True)
  (prob): Softmax(dim=None)
)

In [6]:
# Choose a loss function for training
loss_object = nn.CrossEntropyLoss()

# Train step
def train_step(images, labels, optimizer, model):
    optimizer.zero_grad()
    _,_,_,_,predictions = model(images)
    loss = loss_object(predictions, labels)
    loss.backward()
    optimizer.step()
    return loss.item(), (predictions.argmax(dim=1) == labels).type(torch.float).mean().item()